In [1]:
import rydanalysis
import rydanalysis as ra

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr
from scipy.signal import find_peaks
from scipy.stats import sem
from tqdm.notebook import tqdm
import arc
from pathlib import Path


%matplotlib widget

# Load data

In [2]:
path = Path.cwd().parent
path

WindowsPath('//147.142.18.81/qd-local/qd/rydberg/Projekte - Projects/2020_Aging/2021_01_27/01_SpectrumdDDSred')

In [114]:
data = ra.load_data(path / "raw_data", to_multiindex=True)
images = data.ryd_data.images
images.load()

<xarray.Dataset>
Dimensions:   (shot: 420, x: 256, y: 50)
Coordinates:
  * x         (x) float64 -547.9 -543.6 -539.3 -535.0 ... 539.3 543.6 547.9
  * y         (y) float64 -104.9 -100.6 -96.3 -92.02 ... 92.02 96.3 100.6 104.9
  * shot      (shot) MultiIndex
  - tmstp     (shot) datetime64[ns] 2021-01-27T11:43:06 ... 2021-01-27T12:04:55
  - dDDSred   (shot) float64 3.0 -9.0 -7.0 6.0 2.0 ... 3.0 3.0 -4.0 -7.0 6.0
Data variables:
    image_00  (shot, x, y) float32 206.0 215.0 208.0 214.0 ... 217.0 213.0 213.0
    image_01  (shot, x, y) float32 217.0 218.0 220.0 229.0 ... 376.0 452.0 381.0
    image_02  (shot, x, y) float32 208.0 216.0 217.0 213.0 ... 231.0 227.0 225.0
    image_03  (shot, x, y) float32 229.0 216.0 221.0 225.0 ... 377.0 448.0 369.0
    image_04  (shot, x, y) float32 210.0 196.0 198.0 201.0 ... 203.0 200.0 208.0
    image_05  (shot, x, y) float32 205.0 204.0 200.0 202.0 ... 205.0 206.0 201.0

# Analysis
## Analyse ground state images
### Choose square mask to remove edges of the images

In [115]:
#ground_state_images = images.sel(tCAM=0)

background = images.image_05.mean("shot")
absorption = images.image_01
light = images.image_03
#background

In [116]:
mean_light_image = light.mean("shot")

fig, ax = plt.subplots(figsize=(6, 2))
mean_light_image.T.plot(ax=ax, robust=True)
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [117]:
## Select square mask
### Mask in imaging system 
polygon_mask_indices = [(-500, -90), (500, -90), (500, 90), (-500, 90)]
square_mask = mean_light_image.polygon_mask.get_mask(polygon_mask_indices)
mean_light_image_removed_edges = mean_light_image.where(square_mask)

fig, ax = plt.subplots(figsize=(6, 2))
mean_light_image_removed_edges.T.plot(ax=ax, robust=True);
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
### Simple density calculation without pca

In [119]:
fig, axes = plt.subplots(3, sharex=True)

absorption.mean("shot").where(square_mask).T.plot(ax=axes[0])
light.mean("shot").where(square_mask).T.plot(ax=axes[1])
background.where(square_mask).T.plot(ax=axes[2])
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [120]:
# build_simple imaging

#background = 0  # Set background to 0 because obviously the image sequence was not correct
#background = ground_state_images.image_05.mean("tmstp")

imaging_simple = ra.AbsorptionImaging(absorption - background, mean_light_image - background, t_exp=None, binning=2)
density = imaging_simple.density[2].where(square_mask)

fig, ax = plt.subplots(figsize=(6, 2))
density.T.plot(ax=ax, robust=True);
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [121]:
fig, ax = plt.subplots(figsize=(6, 2))
dDDSlist = np.unique(images.dDDSred)

simple = imaging_simple.density.groupby("dDDSred").mean().where(square_mask)

simple.sel(dDDSred = -10).T.plot(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [122]:
def fit_gs(im):
    model = ra.Gaussian2D()
    params = model.make_params()
    params.add("amp", value=4e12, min=0, max=1e13, vary=True)
    params.add("cen_x", value=-220, vary=True)
    params.add("cen_y", value=50, vary=True)
    params.add("sig_x", value=50, min=0, vary=True)
    params.add("sig_y", value=50, min=0, vary=True) 	
    params.add("offset", value=0 , vary=False) 	
    params.add("theta", value=0, vary=False) 

    fit_result =  model.fit(im, params=params)
    ampl = fit_result.best_values['amp']
    return ampl, fit_result

In [123]:
def get_error(fit_result, param):
    conf = fit_result.conf_interval()[param]
    return conf[4][1] - conf[2][1]

In [131]:
amp_se = pd.Series(dtype=float)
amp_se_err = pd.Series(dtype=float)

for dDDS in dDDSlist[5:15]: 
    ampl, res = fit_gs(simple.sel(dDDSred = dDDS))
    amp_se[dDDS] = ampl

In [132]:
amp_se

-5.0    1.727588e+12
-4.0    2.521422e+12
-3.0    3.414137e+12
-2.0    4.295330e+12
-1.0    4.900498e+12
 0.0    4.743143e+12
 1.0    3.877848e+12
 2.0    1.993474e+12
 3.0    2.101451e+12
 4.0    1.290472e+12
dtype: float64

In [133]:
fig, ax = plt.subplots()
amp_se.plot(ax=ax,marker = 'o')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

In [134]:
from lmfit.models import LorentzianModel

model = LorentzianModel()
params = model.make_params()
params.add('center', value = 0)
params.add('sigma', value = 4 )
params.add('amplitude', value = 1e12 )

fit_result =  model.fit(amp_se.array.to_numpy(), params=params, x = amp_se.index.to_list())
fit_result.plot()
fit_result

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [158]:
ampl, res = fit_gs(simple.sel(dDDSred = -4))
res.plot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(<Figure size 640x640 with 4 Axes>, GridSpec(2, 1, height_ratios=[1, 4]))

In [15]:
# build_simple imaging

background = 0  # Set background to 0 because obviously the image sequence was not correct
# background = ground_state_images.image_05.mean("tmstp")

imaging_simple = ra.AbsorptionImaging(absorption - background, light.mean("tmstp") - background, t_exp=None, binning=2)
density2 = imaging_simple.density.mean("tmstp").where(square_mask)

fig, ax = plt.subplots(figsize=(6, 2))
((density - density2) / density).T.plot(ax=ax, robust=True);
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [18]:
# build_simple imaging

background = 0  # Set background to 0 because obviously the image sequence was not correct
# background = ground_state_images.image_05.mean("tmstp")

imaging_simple = ra.AbsorptionImaging(absorption - background, light.mean("tmstp") - background, t_exp=None, binning=2)
density2 = imaging_simple.density.mean("tmstp").where(square_mask)

fig, ax = plt.subplots(figsize=(6, 2))
((density - density2) / density).sel(y=0, method="nearest").plot(ax=ax);
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

 ## Advanced analysis involving Principle component analysis (PCA)

In [112]:
# Select atom mask

atom_mask = images.elliptical_mask.get_mask(*[-220, 0, 350, 200])

fig, ax = plt.subplots(figsize=(6, 2))
density.where(np.logical_not(atom_mask)).T.plot(ax=ax, robust=True);
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [118]:
pca = light.pca(n_components=1)
no_atoms_mask = np.logical_not(atom_mask)
light_images = pca.find_references(absorption, no_atoms_mask)

ground_state_imaging = ra.AbsorptionImaging(absorption.where(square_mask) - background, light_images - background, t_exp=None, binning=2)

In [119]:
## Select atom mask

atom_mask = images.elliptical_mask.get_mask(*[-220, 0, 350, 200])

fig, ax = plt.subplots(figsize=(6, 2))
ground_state_imaging.density.mean("tmstp").T.plot(ax=ax, robust=True)
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [120]:
## compare both analysis methods

atom_mask = images.elliptical_mask.get_mask(*[-220, 0, 350, 200])

fig, ax = plt.subplots(figsize=(6, 2))
(ground_state_imaging.density.mean("tmstp") - density_removed_edges).T.plot(ax=ax, robust=True);
ax.set_xlabel("x [$\mu$m]")
ax.set_ylabel("y [$\mu$m]")
fig.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

The analysis with pca is problematic for multiple reasons:

- The imaging timing was not correct and we still see some remanences of the shifting on the andor sensor
- The background picture is not useful
- The size of the atom cloud is so large, that the remaining space for pca is too little

The pca tries to avoid these features and does weird things. We will continue without pca

In [124]:
ground_state_imaging = imaging_simple

In [130]:
fig, ax = plt.subplots()

ground_state_imaging.density.where(atom_mask).mean(["x", "y"]).plot(ax=ax)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [162]:
model = ra.Gaussian2D()
params = model.make_params()
params.add("amp", value=12e11, min=0, max=4e11)
params.add("cen_x", value=-244, vary=True)
params.add("cen_y", value=30, vary=True)
params.add("sig_x", value=250, min=0, vary=False)
params.add("sig_y", value=250, min=0, vary=False) 	
params.add("offset", value=0 , vary=True) 	
params.add("theta", value=0, vary=False) 

density = ground_state_imaging.density.mean("tmstp").where(square_mask)
fit_result =  model.fit(density, params=params, method="powell")

In [164]:
fit_result.plot()
fit_result

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Depletion Imaging

In [177]:
def get_rydberg_density(images, tCAM=0.02, do_pca=False):
    background = 0  # images.image_05.where(square_mask).mean("shot")

    gs_images = images.image_01.sel(tCAM=0).where(square_mask) - background
    rydberg_images = images.image_01.sel(tCAM=tCAM, method="bfill").where(square_mask) - background
    
    if do_pca:
        pca = gs_images.pca(n_components=1)
        optimized_gs_images = pca.find_references(rydberg_images, np.logical_not(mask))
    else:
        optimized_gs_images = gs_images.mean("tmstp")
    return -ra.get_density(rydberg_images, optimized_gs_images, saturation_parameter=0)

def fit_rydberg_density(rydberg_density, params=None):
    model = ra.Gaussian2D()
    if params is None:
        params = model.make_params()
        params.add("amp", value=1e11, min=0, max=4e11)
        params.add("cen_x", value=-218, vary=True)
        params.add("cen_y", value=30, vary=True)
        params.add("sig_x", value=100, min=0, vary=True)
        params.add("sig_y", value=50, min=0, vary=True) 	
        params.add("offset", value=0 , vary=True) 	
        params.add("theta", value=0, vary=False) 

    return model.fit(rydberg_density.mean("tmstp"), params=params)

def get_n_Ry(fit_result):
    return 2 * np.pi * np.abs(
        fit_result.best_values['sig_x'] * fit_result.best_values['sig_y']*1e-12 * 
        fit_result.best_values['amp'])

In [251]:
tCAM = np.unique(images.tCAM)[5]

In [252]:
rydberg_density = get_rydberg_density(images, tCAM=tCAM)

In [253]:
fig, ax = plt.subplots()
rydberg_density.mean("tmstp").T.plot(ax=ax)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [254]:
fig, ax = plt.subplots()

rydberg_density.mean("tmstp").sel(x=-233, method="nearest").plot(ax=ax)
ax.set_ylabel("Rydbberg density (integrated along x)")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0, 0.5, 'Rydbberg density (integrated along x)')

In [255]:
fit_result = fit_rydberg_density(rydberg_density, params=None)
n_Ry = get_n_Ry(fit_result)
n_Ry

601.682345047004

In [256]:
fig, (ax0, ax1) = plt.subplots(2, figsize=(12, 8))
fit_result.plot_residuals(ax=ax0)
fit_result.plot_fit(ax=ax1)
fit_result

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [257]:
params = fit_result.params
params["cen_x"].vary = True
params["cen_y"].vary = True
params["sig_x"].vary = True
params["sig_y"].vary = True

In [258]:
nRy_series = pd.Series(dtype=float)
for tCAM, _ in tqdm(list(images.groupby("tCAM"))[2:]):
    rydberg_density = get_rydberg_density(images, tCAM=tCAM)
    fit_result = fit_rydberg_density(rydberg_density, params=params)
    n_Ry = get_n_Ry(fit_result)
    nRy_series[tCAM] = n_Ry

In [259]:
nRy_series.to_csv("n_Ry_depletion.csv")

In [260]:
fig, ax = plt.subplots()

nRy_series.plot(ax=ax)
# ax.set_ylim(0, 13000)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:>

Compare to counts

In [23]:
peak_df = data.scope_traces.peaks_summary.get_peak_description(width=2e-9, height=0.0007, distance=None)

In [24]:
peak_df.to_csv("peak_df.csv")

In [208]:
peak_df = pd.read_csv("peak_df.csv", index_col=[0, 1, 2])
nRy_series = pd.read_csv("n_Ry_depletion.csv", index_col=0, squeeze=True)

In [209]:
counts = ra.summarize_peak_description(peak_df).counts
ion_int = data.scope_traces.peaks_summary.integrate(height = 0.0007)
# ion_int.load()

In [210]:
fig, ax = plt.subplots()

# plot ion counts

groupby = counts.groupby("tCAM")
detection_efficiency = 0.32
ax.errorbar(groupby.mean().tCAM*1e3, groupby.mean()/detection_efficiency, groupby.reduce(sem), label=r"ion count / {}%".format(detection_efficiency * 100),
            linestyle='', marker='o')

# plot ion integrated

groupby = ion_int.groupby("tCAM")
#ax.errorbar(groupby.mean().tCAM*1e3, groupby.mean()/0.02, groupby.reduce(sem), label="ion integrated / 0.48%",
#            linestyle='', marker='.')

# plot depletion imaging
ax.plot(nRy_series.index*1e3, nRy_series, label="depletion imaging",
            linestyle='', marker='o')

ax.legend()
ax.set_xlabel(r"time [$\mu$s]")
ax.set_ylabel("Rydberg number")
fig.savefig("depletion.png", dpi=300, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [250]:
fig, ax = plt.subplots()

# plot ion counts
counts = ra.summarize_peak_description(peak_df).counts
groupby = counts.groupby("tCAM")
ax.errorbar(nRy_series, groupby.mean()[2:], groupby.reduce(sem)[2:],
            linestyle='', marker='o')
ax.plot([0, 1000], [0, 400])
ax.set_xlabel(r"rydberg (depletion imaging)")
ax.set_ylabel("rydberg (ions)")
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
fig.savefig("depletion_ion_vs_depletion.png", dpi=300, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [248]:
exp_sat_model = ra.ExpSatModel()
exp_sat_params = exp_sat_model.make_params()
exp_sat_params.add("amp", value=350, min=0, vary=True)
exp_sat_params.add("offset", value=0, vary=False)
exp_sat_params.add("tau", value=1000, min=0, vary=True)
exp_sat_fit = exp_sat_model.fit(groupby.mean()[2:], x=nRy_series, params=exp_sat_params)
exp_sat_fit

In [249]:
fig, ax = plt.subplots()

ax.errorbar(nRy_series, groupby.mean()[2:], groupby.reduce(sem)[2:],
            linestyle='', marker='o')
x = np.linspace(0, max(nRy_series))
ax.plot(x, exp_sat_fit.eval(x=x))
ax.set_xlabel(r"rydberg (depletion imaging)")
ax.set_ylabel("rydberg (ions)")
ax.set_xlim(left=0)
ax.set_ylim(bottom=0)
fig.savefig("try_exp_fit.png", dpi=300, bbox_inches="tight")

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …